#handmaded_1layer_MLP

Still some bugs when batch_size != 1...

In [2]:
import numpy as np
from itertools import product

In [3]:
import gzip, pickle
f = gzip.open('data/mnist.pkl.gz')

data = pickle.load(f)

train = data[0][0]
train_y =data[0][1]

valid = data[1][0]
valid_y = data[1][1]

test = data[2][0]
test_y = data[2][1]

In [4]:
#Activation functions and their derivatives. maded to take a vector in input.
def rect(v):
    return np.maximum(np.zeros(v.shape),v);

def rect_prime(v):
    tmp = np.zeros(v.shape[0])
    for i in xrange(v.shape[0]):
        if( v[i] < 0):
            tmp[i] = 0
        else:
            tmp[i] = 1
    return tmp

rect.prime = rect_prime


def softmax(a):
    """ Stable implementation on softmax function """
    a_max = np.max(a)
   
    n = 0 # normalisation_factor
    s = np.zeros(len(a))
   
    for i, x in enumerate(a):
        s[i] = np.exp(x-a_max)
        n += s[i]
       
    return 1.0/n * s

def linear(a):
    return a

def linear_prime(a):
    return np.ones(a.shape[0])

def sigmoid(a): #needs to be vectorized
    return 1.0/(1+np.exp(-a))

def sigmoid_prime(a): #needs to be vectorized
    x = sigmoid(a)
    return x*(1-x)

sigmoid.prime = sigmoid_prime
def tanh(a):#needs to be vectorized
    return (np.exp(2*a) - 1)/(np.exp(2*a) + 1)

#needs to be vectorized
tanh.prime = lambda a : 1.0 - (tanh(a))**2

#Costs functions for supervised learning
def L1(x,y):
    pass

def L2(x,y):
    assert x.size == y.size
    return np.sum(np.abs(x-y))

def EM(x,y):
    return -1.0*np.log(x[np.where(y==1)])

#utility
def onehot(m, y):
    result = np.zeros(m)
    result[y] = 1
    return result

def chunker(seq, size):
    #Taken from http://stackoverflow.com/questions/434287/what-is-the-most-pythonic-way-to-iterate-over-a-list-in-chunks
    return (seq[pos:pos + size] for pos in xrange(0, len(seq), size))

def return_if_null(a,b):
    """ if a is None then return b else do return a"""
    if a is None:
        return b
    return a

print rect(np.array([3,4,5,6,-2, 0.2, -9]))
print rect.prime(np.array([3,4,5,6,-2, 0.2, -9]))
print softmax(np.array([1,1,1]))

[ 3.   4.   5.   6.   0.   0.2  0. ]
[ 1.  1.  1.  1.  0.  1.  0.]
[ 0.33333333  0.33333333  0.33333333]


In [5]:
class MLP:
    """ 'handmade' 1 hidden layer MLP"""
    def __init__(self,nb_input,nb_hidden,nb_output,hidden_func=rect,output_func=softmax, cost_func=EM
                 ,init_dist= lambda i,j: np.random.uniform(low=-1.0,high=1.0) ):
        
        self.W1= np.random.uniform(high=1,low=1,size= (nb_input, nb_hidden))
        self.W2= np.random.uniform(high=1,low=1,size= (nb_hidden,nb_output))
        #self.W2= np.fromfunction(init_dist, (nb_hidden, nb_output), dtype=np.float)
        self.b1= np.zeros(nb_hidden,dtype=np.float)
        self.b2= np.zeros(nb_output,dtype=np.float)
        self.hidden_func = hidden_func
        self.output_func = output_func
        self.cost_func = cost_func
        
    def train(self,data,data_labels, batch_size = None,nb_epoch=1):
        if batch_size == None:
            batch_size = len(data)
        
        for i in range(nb_epoch):
            if batch_size == 1:

                for i in range(len(data)):
                    grad_b1, grad_W1, grad_b2, grad_W2 = self.backprop(data[i],data_labels[i])
                    self.update_parameters(grad_b1, grad_W1, grad_b2, grad_W2)

                
            else :
                grad_b1 = np.zeros((batch_size,self.b1.shape[0]),dtype=np.float)
                grad_b2 = np.zeros((batch_size,self.b2.shape[0]),dtype=np.float)
                grad_W1 = np.zeros((batch_size,self.W1.shape[0],self.W1.shape[1]),dtype=np.float)
                grad_W2 = np.zeros((batch_size,self.W2.shape[0],self.W2.shape[1]),dtype=np.float)

                for i,j in zip(chunker(data,batch_size),chunker(data_labels,batch_size)):                
                    #  print(i,j)
                    y=0
                    #print(i)
                    for x in i:
                        grad_b1[y], grad_W1[y], grad_b2[y], grad_W2[y] = self.backprop(x,j)
                        y += 1
                    
                    if len(i) == batch_size:
                        self.update_parameters(np.mean(grad_b1), np.mean(grad_W1), np.mean(grad_b2), np.mean(grad_W2))
                    else :
                        b = len(i)
                        assert b == y
                        self.update_parameters(np.mean(grad_b1[:b]), np.mean(grad_W1[:b]), np.mean(grad_b2[:b]), np.mean(grad_W2[:b]))


    def compute_predictions(self,data):   
        return np.array([self.compute_prediction(data[i]) for i in range(len(data))])

    def compute_prediction(self,x):
        return np.argmax(self.frontprop(x,self.b1,self.W1,self.b2,self.W2)[3])
        
    def frontprop(self,x,b1,W1,b2,W2):
        ha = np.dot(x,W1) + b1
        #ha = np.dot(W1,x) + b1
        
        hs = self.hidden_func(ha)         
        oa = np.dot(hs,W2) + b2
        os = self.output_func(oa)
        #np.sum(-self.t*np.log(self.y)
        #L = -1.0*np.log(os[y])
        return ha,hs,oa,os
        
    def backprop(self,x,y):
        """
        Computes frontprop and backprop
        self.grad_b2 = grad_oa   
        self.grad_b1 = self.grad_ha
        """
        
        ha,hs,oa,os = self.frontprop(x,self.b1,self.W1,self.b2,self.W2)
        #print(ha.shape,hs.shape,oa.shape,os.shape)
    
        grad_oa = os - onehot(self.b2.shape[0], y)
        #grad_W2 = np.outer(grad_oa,hs)
        grad_W2 = np.outer(hs,grad_oa)
        #grad_hs = np.dot(self.W2, grad_oa )
        grad_hs = np.dot(self.W2, grad_oa )
        
        grad_ha = np.multiply(self.hidden_func.prime(ha), grad_hs )
        #grad_W1 = np.outer(x,grad_ha)
        grad_W1 = np.outer(x,grad_ha)
        
        #grad_x = np.dot( self.W1.T, grad_ha)
        #  print(grad_W1.shape, grad_ha.shape, grad_hs.shape,grad_W2.shape, grad_oa.shape)
        return grad_ha, grad_W1, grad_oa, grad_W2
    
    def update_parameters(self,grad_b1,grad_W1,grad_b2,grad_W2,learning_rate=0.1):
        #print(self.b1,grad_b1)
        self.b1 -= learning_rate * grad_b1
        self.W1 -= learning_rate * grad_W1
        self.b2 -= learning_rate * grad_b2
        self.W2 -= learning_rate * grad_W2
    
    def printparameters(self):
        print("b1 : "+str(self.b1)+" W1 : "+str(self.W1))
        print("b2 : "+str(self.b2)+" W2 : "+str(self.W2))     
    
    def grad_diff(self, x,y, epsi = 0.000001):
        """Gradient finite difference. for debugging purposes."""
        grad_b1 = np.zeros(self.b1.shape[0])
        grad_b2 = np.zeros(self.b2.shape[0])
        grad_W1 = np.zeros(self.W1.shape)
        grad_W2 = np.zeros(self.W2.shape)


        yy = onehot(self.b2.shape[0], y)

        cost = self.cost_func(self.frontprop(x,self.b1,self.W1,self.b2,self.W2)[3],yy)
        gb1,gw1, gb2, gw2 = self.backprop(x,y)
        
        # compare b1
        b1 = np.array(self.b1)
        for i in xrange(self.b1.shape[0]):  
            b1[i] -= epsi
            os2 = self.frontprop(x,b1,self.W1,self.b2,self.W2)[3]
            cost2 = self.cost_func(os2,yy)

            grad_b1[i] = (cost - cost2)/epsi
            b1[i] += epsi

        print "\nValeur grad_b1", gb1
        print "\nValeur par différence finit", grad_b1

        # compare b2
        b2 = np.array(self.b2)
        for i in xrange(self.b2.shape[0]):  
            b2[i] -= epsi
            os2 = self.frontprop(x,self.b1,self.W1,b2,self.W2)[3]
            cost2 = self.cost_func(os2,yy)
            grad_b2[i] = (cost - cost2)/epsi
            b2[i] += epsi

        print "\nValeur grad_b2", gb2
        print "\nValeur par différence finit", grad_b2

        # compare W1
        W1 = np.array(self.W1)
        for i,j in product(xrange(self.W1.shape[0]), xrange(self.W1.shape[1])):
            W1[i][j] -= epsi
            os2 = self.frontprop(x,self.b1,W1,self.b2,self.W2)[3]
            cost2 = self.cost_func(os2,yy)
            grad_W1[i][j] = (cost - cost2)/epsi
            W1[i][j] += epsi

        print "\nValeur grad_W1", gw1
        print "\nValeur par différence finit", grad_W1

        # compare W2
        W2 = np.array(self.W2)
        for i,j in product(xrange(W2.shape[0]), xrange(W2.shape[1])):
            W2[i][j] -= epsi
            os2 = self.frontprop(x,self.b1,self.W1,self.b2,W2)[3]
            cost2 = self.cost_func(os2,yy)
            grad_W2[i][j] = (cost - cost2)/epsi
            W2[i][j] += epsi

        print "\nValeur grad_W2", gw2
        print "\nValeur par différence finit",grad_W2
        
        
    

In [6]:
nb_input = len(train[0])
nb_output = 10
nb_hidden = 200
model = MLP(nb_input,nb_hidden,nb_output)
model.train(train,train_y,1)

#def __init__(self,nb_input,nb_hidden,nb_output,hidden_layer_func=rect,output_func=softmax, cost_func=L2                 ,init_dist= lambda i,j: np.random.uniform(low=-1.0,high=1.0) ):

        
# def train(self,data, batch_size = None,nb_epoch=1):

In [ ]:
model.grad_diff(test[0],test_y[0])

In [7]:

test_prediction = model.compute_predictions(test)
print(test_prediction)
print(test_y)
errors = 0
nb_test = len(test_y)
for i in range(nb_test):
    if test_prediction[i] != test_y[i]:
        errors += 1
print('tx d\'erreurs',float(errors)/nb_test)


[0 0 0 ..., 0 0 0]
[7 2 1 ..., 4 5 6]
("tx d'erreurs", 0.902)


In [ ]:
print(model.W2)


In [ ]:

# Retourne la valeur de L en fonction des matrices données
class fprop:
    def __init__(self, W1,W2,b1,b2,x,y):
        self.W1=W1
        self.W2=W2
        self.b1=b1
        self.b2=b2
        self.x=x
        self.y=y       
        self.m = b2.shape[0]
        self.dh = b1.shape[0]
        
    def compute(self):
        self.ha = np.dot(self.W1, self.x) + self.b1
        self.hs = rect(self.ha)
        self.os = softmax(np.dot(self.W2, self.hs) + self.b2)
        self.L = -1.0*np.log(self.os[self.y])
        return self.L
    
    def compute_predictions(self, data):
        self.ha = np.dot(self.W1, data) + self.b1
        self.hs = rect(self.ha)
        self.os = softmax(np.dot(self.W2, self.hs) + self.b2)
        return np.argmax(self.os)
    
#calcul des gradients
class bprop:
    def __init__(self, fprop):
        self.fprop = fprop
        
        grad_oa = fprop.os - onehot(fprop.m, fprop.y)
        self.grad_b2 = grad_oa   
        self.grad_W2 = np.outer( grad_oa, fprop.hs)
        self.grad_hs = np.dot(fprop.W2.T, grad_oa )
        self.grad_ha = np.multiply(rectPrime(fprop.ha), self.grad_hs )
        self.grad_b1 = self.grad_ha
        self.grad_W1 = np.outer( self.grad_ha, fprop.x)
        self.grad_x = np.dot( fprop.W1.T, self.grad_ha)
        #print(grad_ha.shape, grad_hs.shape, grad_oa.shape)

        

# calcul du gradient par différence finit
def gradDiff(bprop):
    epsi = 0.000001
    
    grad_b1 = np.zeros(bprop.fprop.b1.shape[0])
    grad_b2 = np.zeros(bprop.fprop.b2.shape[0])
    grad_W1 = np.zeros(bprop.fprop.W1.shape)
    grad_W2 = np.zeros(bprop.fprop.W2.shape)

    
    # compare b1
    L = bprop.fprop.L
    for i in xrange(bprop.fprop.b1.shape[0]):  
        bprop.fprop.b1[i] -= epsi
        bprop.fprop.compute()
        grad_b1[i] = (L - bprop.fprop.L)/epsi
        bprop.fprop.b1[i] += epsi
        
    print "\nValeur grad_b1"
    print bprop.grad_b1
    print "Valeur par différence finit"
    print grad_b1
   
    # compare b2
    bprop.fprop.compute()
    L = bprop.fprop.L
    for i in xrange(bprop.fprop.b2.shape[0]):  
        bprop.fprop.b2[i] -= epsi
        bprop.fprop.compute()
        grad_b2[i] = (L - bprop.fprop.L)/epsi
        bprop.fprop.b2[i] += epsi

    print "\nValeur grad_b2"
    print bprop.grad_b2
    print "Valeur par différence finit"
    print grad_b2
        
    
    # compare W1
    bprop.fprop.compute()
    L = bprop.fprop.L
    for i in xrange(bprop.fprop.W1.shape[0]):
        for j in xrange(bprop.fprop.W1.shape[1]):
            bprop.fprop.W1[i][j] -= epsi
            bprop.fprop.compute()
            grad_W1[i][j] = (L - bprop.fprop.L)/epsi
            bprop.fprop.W1[i][j] += epsi
            
    print "\nValeur grad_W1"
    print bprop.grad_W1
    print "Valeur par différence finit"
    print grad_W1

    
    # compare W2
    bprop.fprop.compute()
    L = bprop.fprop.L
    for i in xrange(bprop.fprop.W2.shape[0]):
        for j in xrange(bprop.fprop.W2.shape[1]):
            bprop.fprop.W2[i][j] -= epsi
            bprop.fprop.compute()
            grad_W2[i][j] = (L - bprop.fprop.L)/epsi
            bprop.fprop.W2[i][j] += epsi
            
    bprop.fprop.compute()
    
    print "\nValeur grad_W2"
    print bprop.grad_W2
    print "Valeur par différence finit"
    print grad_W2
